##Return on Investment (ROI)

To estimate the Return on Investment (ROI) for property improvements using an ANN, we need to set up a structured process. Here’s a step-by-step approach:

Step 1: Define the ROI Variable
Since ROI isn’t likely to be present directly in the dataset, you would typically calculate it based on:

Historical data on property improvements: If available, use the increase in SalePrice after certain improvements to calculate ROI.
Expert assumptions: Define typical ROI based on common property improvement projects (e.g., adding a bedroom, remodeling bathrooms) using real estate industry standards.
For simplicity, let’s assume we have historical data that allows us to define ROI as follows:

ROI
=
Expected SalePrice Increase
Improvement Cost
ROI=
Improvement Cost
Expected SalePrice Increase
​

For this example, we’ll create a synthetic ROI column based on some feature combinations and hypothetical values.

Step 2: Prepare the Dataset for ROI Prediction
We’ll select features relevant to estimating ROI, such as:

Property Value Features: TotalValue, BuildingValue, LandValue
Property Characteristics: Bedrooms, FullBath, HalfBath, Acreage, YearBuilt
We’ll then preprocess the data and train an ANN to predict ROI.

Step 3: Build and Train the ANN Model
We’ll design a regression ANN model with a linear output layer to predict ROI as a continuous variable.







Explanation of Key Steps
Synthetic ROI Generation: We generated a hypothetical ROI based on the expected increase in SalePrice relative to an assumed improvement cost.
Feature Selection: Relevant features were chosen to capture the impact of property characteristics and value components on ROI.
ANN Model: A regression model with a linear output layer predicts ROI as a continuous value.
Model Evaluation: We evaluate the model on the test set to check the accuracy in predicting ROI.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Load data
nashville_data = pd.read_csv('Nashville_Housing_ML.csv')

# Step 1: Create a synthetic ROI column for demonstration purposes
nashville_data['ImprovementCost'] = nashville_data['Bedrooms'] * 10000 + nashville_data['FullBath'] * 5000
nashville_data['ExpectedSalePriceIncrease'] = nashville_data['BuildingValue'] * 0.1  # Hypothetical increase
nashville_data['ROI'] = nashville_data['ExpectedSalePriceIncrease'] / nashville_data['ImprovementCost']

# Drop rows where ImprovementCost might be zero to avoid division errors
nashville_data = nashville_data[nashville_data['ImprovementCost'] > 0]

# Convert 'YearBuilt' to a datetime format and extract the year
nashville_data['YearBuilt'] = pd.to_datetime(nashville_data['YearBuilt'], errors='coerce').dt.year

# Drop any remaining rows with NaN values in 'YearBuilt'
nashville_data = nashville_data.dropna(subset=['YearBuilt'])

# Select relevant features for predicting ROI
selected_features = ['TotalValue', 'BuildingValue', 'LandValue', 'Bedrooms', 'FullBath', 'HalfBath', 'Acreage', 'YearBuilt']
X = nashville_data[selected_features].values
y = nashville_data['ROI'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Build the ANN model
ann = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')  # Linear output for regression
])

# Compile the model with a smaller learning rate
ann.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Train the model
ann.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping])

# Step 4: Evaluate the model and make predictions
test_loss, test_mae = ann.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# Predict ROI on the test set
predictions = ann.predict(X_test)

# Display some sample predictions
for i in range(5):
    print(f"Predicted ROI: {predictions[i][0]:.2f}, Actual ROI: {y_test[i]:.2f}")


<ipython-input-2-87f24e8b15fc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nashville_data['YearBuilt'] = pd.to_datetime(nashville_data['YearBuilt'], errors='coerce').dt.year


Epoch 1/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0594 - mae: 0.1562
Epoch 2/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0217 - mae: 0.0651
Epoch 3/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0076 - mae: 0.0379
Epoch 4/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0055 - mae: 0.0306
Epoch 5/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0130 - mae: 0.0296
Epoch 6/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0039 - mae: 0.0232
Epoch 7/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0076 - mae: 0.0241
Epoch 8/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0052 - mae: 0.0208
Epoch 9/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0061 - mae: 0.0218
Epoch 10/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0047 - mae: 0.0186
Epoch 11/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0035 - mae: 0.0182
Epoch 12/50
631/631 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0023 - mae: 0.0168
Epoch 13/50
631/631 ━━━━━